# Imports

In [3]:
%cd C:/Users/nkalimip/Downloads/sentinelsat-main/sentinelsat-main

C:\Users\nkalimip\Downloads\sentinelsat-main\sentinelsat-main


1.  Parse csv file to extract ALL image names, even multiple entries
        parse data as csv
        extract the column as a list of lists
        flatten the lixt of lists into one big list
        remove null none 
2.  make pandas df from clean list with one column correspondng to input image name
3.  write function to search for the input image name in gee and return system:id , put in new column of pandas df
        start gee api
        in python for loop of df column of image names
            use gee command to filter the S2 collection to get image 
            real_name = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filter(ee.Filter.equals("system:index","20210719T190919_20210719T191700_T10TER")) \
                         .first() \
                         .get('PRODUCT_ID') \
                         .getInfo()
4.  iterate over list of real names
    

In [10]:
#import libraries
import ee
import pandas as pd
import os

# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

# sentinelsat requirements
# requests
# click >= 7.1
# html2text
# geojson >= 2
# tqdm >= 4.58
# geomet

from datetime import date

C:\Users\nkalimip\Anaconda3\envs\simha1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#Autheticate and initillize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWtgzh4vTz6cdQUhIh3JN3uMq4TU_wXRO-Bg_IPpymp3y5dY4Bz2vgfXJIc



Successfully saved authorization token.


# Directory Function

In [4]:
def directory1(directory):
    """Gets the names of a directory"""
    entries = os.listdir(directory)
    return entries
    # print(entries)  

# Cleans CSV lists Function

In [5]:
def list_splitter(data,name):
    """cleans/splits df of list of images"""
    
    # Input data
    df = pd.read_csv(data)
    # print(df)
    
    # Change type to string, remove [], send to list
    df['Images'] = df['Images'].astype(str)
    # print(df['Images'])
    df['Images'] = df['Images'].map(lambda x: x.lstrip('[').rstrip(']'))
    list_of_img =  df["Images"].tolist() 
    # print(list_img)
    
    # Clean list without nan
    cleanedList = [x for x in list_of_img if str(x) != 'nan']
    
    # Print list
    # print("list1")
    # print(cleanedList)
    
    # print("#---------------------------")
    # print(len(cleanedList))
    
    # Arrays
    cleanedList2  = []
    list_to_add = []
    
    # Find list images containing more than 1 img
    for counter0 in range(0,len(cleanedList)):
        count_comma = cleanedList[counter0].count(',')
        # print(count_comma)
       
        if count_comma>0:
            list_multiple  = cleanedList[counter0]
            # print(list_multiple)
            cleanedList2.append(list_multiple)
            
    # Find list images containing 1 img  
    cleanedList3 = [item for item in cleanedList if item not in cleanedList2]
    
    # Print
    # print("clean list2")
    # print(cleanedList2)
    # print("clean list3")
    # print(cleanedList3)

    # Split the data into separate imgs 
    for counter1 in range(0,len(cleanedList2)):
        list_item =cleanedList2[counter1] 
        # print(list_item)
        mylist = list_item.split(',')
        for counter2 in range(0,len(mylist)):
            withoutspace  = mylist[counter2].replace(" ","")
            
            list_to_add.append(withoutspace)
    
    
    # print("list to add")
    # print(list_to_add)
    
    
    # add two lists      
    list_fin = list_to_add+cleanedList3
    
    # create dataframe
    newdf= pd.DataFrame(list_fin,columns=["Images__clean"])
    newdf = newdf.drop_duplicates()
    
    newdf["neon_sites"] = name

    # newdf.to_csv("/home/richard_fernandes/Downloads/data1.csv")
    
    print("______________________________________________")
    return newdf

    
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/ABBY_kmlwithImages.csv"))
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/DSNYCLEARIMAGES2.csv"))
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/UNDECLEARIMAGES.csv"))

# Real Name Function

In [6]:
def real_name(short_name):
    """ function to help find real names of files"""
    real_name = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filter(ee.Filter.equals("system:index",short_name)) \
                    .first() \
                    .get('PRODUCT_ID') \
                    .getInfo()

    return real_name

In [40]:
# print(real_name("20210719T190919_20210719T191700_T10TER"))

S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339


# Search string Function

In [7]:
def search_string(string1):
    """ searchs esa datahub for image and downsloads it """
    
    # Connect to the API
    api = SentinelAPI('****', '*******', 'https://apihub.copernicus.eu/apihub')

    # Split string 
    string = string1 
    split_s=string.split("_")
    
    # Built a file string
    sat_name = "S2A"+"_"
    rest_name = split_s[1]+"_"+split_s[2]+"_"+split_s[3]+"_"+split_s[4]+"_"+split_s[5]+"_"+split_s[6]
    safe_name = '.SAFE'
    # # wc = '_*_'

    #try and except to find errors
    try:
        print("#--------Try---------------------------------------")
        sat_name = 'S2A_'
        
        # Find and query image
        filename1 = sat_name+rest_name+safe_name
        # print(filename1)

        query1 = api.query(filename = filename1)
        # print(query1)
        
        # download information
        api.download_all(query1)
        result = api.download_all(query1)
        # print(result)
        
        #check if something is downloaded
        if str(result) =="ResultTuple(downloaded={}, retrieval_triggered={}, failed={})":
            raise ValueError('Nothing was dowloaded')

    except:
        print("#--------Except---------------------------------------")
        sat_name ='S2B_'
        
        # Find and query image
        filename1 = sat_name+rest_name+safe_name
        # print(filename1)
        
        query1 = api.query(filename = filename1)
        # # print(query1)
        
        # download information
        api.download_all(query1)
        result = api.download_all(query1)

    # print(result)
# 
# print(search_string("S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339"))

# Main function

In [12]:
# List of csvs in directory
def main_sentinel_sat_downloader(directory):
    basic_directory = directory
    directory_lib = basic_directory+"2_sentinel2_clear_sky_images"
    list_directory = directory1(directory_lib)
    # print(list_directory)
    df = pd.DataFrame()


    # Looping through csvs
    for counter0 in range(0,len(list_directory)): 
        # Clean lists of imgs in csv
        newdf = list_splitter(directory_lib +"/"+list_directory[counter0],list_directory[counter0])
        # print(newdf)

        # Search every Image
        for counter in range(0,len(newdf.index)): 
            if ~newdf.empty:

                df = pd.concat([df, newdf])

                # Run through process
                # print("p")

                # name_stage0 = newdf['Images__clean'][counter]
                # print(name_stage0)

                # Finds real name of img
                # name_stage1 = real_name(name_stage0)

                # search and download img
                # search_string(name_stage1)
                # # print(name_stage1)


    #get id of duplicate and reset index
    df = df.drop_duplicates()
    df = df.reset_index()
    df["true_name"] = df["Images__clean"].apply(real_name)
    # print(df)
    df.head()
    df.to_csv(basic_directory + "/3_images_clean/images_clean.csv")

    # #loop for finding
    # for counter in range(0,len(df.index)):
    #     name_stage0 = df['Images__clean'][counter]
        #print(df['Images__clean'][counter])

    #     # Finds real name of img
    #     name_stage1 = real_name(name_stage0)

    #     # search and download img
    #     search_string(name_stage1)
    #     # # print(name_stage1)



In [ ]:
main_sentinel_sat_downloader( "C:/Users/nkalimip/Downloads/downloading_process/")

______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
